In [1]:
import numpy as np # linear algebra
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import sklearn
import sklearn.metrics

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from math import sqrt

In [3]:
#building all kinds of evaluating parameters
from sklearn.metrics import classification_report, accuracy_score  
from sklearn.metrics import precision_score, recall_score 
from sklearn.metrics import roc_curve,auc, roc_auc_score, average_precision_score
from sklearn.metrics import f1_score, matthews_corrcoef 
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import confusion_matrix 

In [4]:
from imblearn.pipeline import make_pipeline as make_pipeline_imb # To do our transformation in a unique time
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from sklearn.model_selection import train_test_split
classifier = RandomForestClassifier

In [5]:
import pandas as pd

url2 = 'Dataset/PRFI_4M_DEF_HEAD.txt'
df = pd.read_table(url2, sep=';')

In [6]:
df["FLAG_FRODE"].replace({"T": 1, "F": 0}, inplace=True)

In [7]:
df["DT_OP_GMT"] = pd.to_datetime(df["DT_OP_GMT"])

start_datetr = '2016-03-01'
end_datetr = '2016-05-31'

mask_tr = (df['DT_OP_GMT'] >= start_datetr) & (df['DT_OP_GMT'] < end_datetr) 
df_train = df.loc[mask_tr]

print(df_train)

            ID_OPER  DT_OP_GMT ORA_OP_GMT COD_CANA  COD_CABC  COD_ESITO  \
0        6231261973 2016-03-14   11:27:28      ATM      6011          0   
1        6217512045 2016-03-09   09:48:09      INT      5942          0   
2        6204998318 2016-03-04   12:01:47      INT      5192          0   
3        6260986375 2016-03-25   08:29:03      INT      5968          0   
4        6214969493 2016-03-08   10:36:09      POS      5411          0   
...             ...        ...        ...      ...       ...        ...   
5438812  6443141702 2016-05-30   19:07:33      INT      7273          0   
5439277  6442518610 2016-05-30   15:57:05      INT      5045          0   
5439283  6442463929 2016-05-30   17:44:10      INT      8999          0   
5439406  6442771077 2016-05-30   16:54:51      ATM      6011          0   
5439830  6396976594 2016-05-13   17:07:03      ATM      6011          0   

         PROCESSING_COD  IMP_TRZ_RECON FL_AUTORIZZAZIONE COD_ABC_PAES  ...  \
0                3661

In [8]:

start_datets = '2016-06-07'
end_datets = '2016-06-30'


#greater than the start date and smaller than the end date
mask = (df['DT_OP_GMT'] >= start_datets) & (df['DT_OP_GMT'] < end_datets)
df_test = df.loc[mask]


In [9]:
df_train = df_train[['ID_OPER', 'DT_OP_GMT', 'ORA_OP_GMT', 'COD_CANA', 'COD_CABC',
                'PROCESSING_COD', 'IMP_TRZ_RECON', 'FL_AUTORIZZAZIONE',
                'COD_ABC_PAES', 'ID_TIPO_CARTA', 'TIPO_INPUT_MODE',
                'COD_CIRC', 'FLAG_FRODE', 'TRX_TYPE', 'Row_Num', 'TIME_ORA_OP','DEV_MEDIA_IMP_TC_1M','PERC_RISK_CIRC_TC_TO_1M',
               'DEV_MEDIA_IMP_CAN_1M','DEV_MEDIA_IMP_TO_1M','PERC_OPER_NON_AUTOR_1M','PERC_OPER_NON_AUTOR_2M','MEDIA_NON_AUT_SEC_4',
               'DEV_IMP_MEDIA_ULTIME_3', 'PERC_RISK_MCC_1M','FLAG_IMP_PREC_0','TREND_3_5','FLAG_CAMBIO_PAESE_AUT',
               'FLAG_CAMBIO_PAESE_NON_AUT','PERC_ESITO_NSF_G','PERC_ESITO_CV2_G',
               'PERC_ESITO_PIN_G','PERC_ESITO_SCUDO_G','PERC_ESITO_DT_SCAD_G','PERC_ESITO_ALTRO_G',
               'DEV_MEDIA_AUT_SEC_4']]

In [10]:
from sklearn.preprocessing import RobustScaler

rob_scaler = RobustScaler()

df_train['IMP_TRZ_RECON_NORM'] = rob_scaler.fit_transform(df_train['IMP_TRZ_RECON'].values.reshape(-1,1)) 

df_train.drop(['IMP_TRZ_RECON'], axis=1, inplace=True)

In [11]:
# Number of data points in the minority class
df_train = df_train.reset_index()
number_records_fraud = len(df_train[df_train.FLAG_FRODE == 1])
fraud_indices = np.array(df_train[df_train.FLAG_FRODE == 1].index)

In [12]:
# Picking the indices of the normal classes
normal_indices = df_train[df_train.FLAG_FRODE == 0].index

# Out of the indices we picked, randomly select "x" number (number_records_fraud)
random_normal_indices = np.random.choice(normal_indices, number_records_fraud, replace = False)
random_normal_indices = np.array(random_normal_indices)

# Appending the 2 indices
under_sample_indices = np.concatenate([fraud_indices,random_normal_indices])

# Under sample dfset
df_train_undersample = df_train.iloc[under_sample_indices,:]

#X_undersample = under_sample_df.loc[:, under_sample_df.columns != 'FLAG_FRODE']
#y_undersample = under_sample_df.loc[:, under_sample_df.columns == 'FLAG_FRODE']
df_train_undersample

,index,ID_OPER,DT_OP_GMT,ORA_OP_GMT,COD_CANA,COD_CABC,PROCESSING_COD,FL_AUTORIZZAZIONE,COD_ABC_PAES,ID_TIPO_CARTA,...,FLAG_CAMBIO_PAESE_AUT,FLAG_CAMBIO_PAESE_NON_AUT,PERC_ESITO_NSF_G,PERC_ESITO_CV2_G,PERC_ESITO_PIN_G,PERC_ESITO_SCUDO_G,PERC_ESITO_DT_SCAD_G,PERC_ESITO_ALTRO_G,DEV_MEDIA_AUT_SEC_4,IMP_TRZ_RECON_NORM
36,38,6226002651,2016-03-12,09:16:37,INT,7399,360000,T,BMU,1009,...,1,0,0.0,0.0,0.0,0.0,0.0,0.0,-1.000000,-0.319642
38,40,6251120205,2016-03-21,16:55:41,INT,5968,0,T,GBR,1028,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.061449,-0.171264
60,71,6277642583,2016-03-31,19:43:54,INT,5499,6300,T,GBR,1028,...,1,0,0.0,0.0,0.0,0.0,0.0,0.0,2.039607,-0.308585
101,112,6197847455,2016-03-02,01:47:32,INT,7995,116300,T,MLT,1028,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2052.826531,0.786003
102,113,6197847677,2016-03-02,01:46:43,INT,7995,116300,T,MLT,1028,...,1,0,0.0,0.0,0.0,0.0,0.0,0.0,1057.342105,0.786003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2854974,3436004,6263368775,2016-03-25,18:19:20,POS,5311,0,T,ITA,1063,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,37.997611,-0.126154
358066,419660,6229927885,2016-03-13,18:49:07,POS,5311,0,T,ITA,1017,...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,-0.822842,0.020897
3665380,4416940,6235176101,2016-03-15,17:01:02,POS,5411,0,T,ITA,1055,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,18.418458,-0.255625
961774,1150455,6270921001,2016-03-29,13:25:54,ATM,6011,16100,T,ITA,1017,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,-0.293950,4.655758


In [13]:
df_test

,ID_OPER,DT_OP_GMT,ORA_OP_GMT,COD_CANA,COD_CABC,COD_ESITO,PROCESSING_COD,IMP_TRZ_RECON,FL_AUTORIZZAZIONE,COD_ABC_PAES,...,PERC_ESITO_ALL_G,PERC_ESITO_ALL_M,DEV_MEDIA_AUT_SEC_4,DEV_MEDIA_AUT_TO_SEC_4,MEDIA_NON_AUT_SEC_4,MEDIA_NON_AUT_SEC_3,MEDIA_NON_AUT_SEC_2,TIME_NON_AUT_SEC_LAST,ANNO,MESE
35,6469120910,2016-06-08,22:00:16,INT,5968,0,0,84.99,T,LUX,...,0.0,0.000000,-1.000000,0.000000,305481.50,2.038930e+05,305839.0,15948784,2016,6
47,6490500262,2016-06-16,11:20:09,ATM,6011,0,16100,20.00,T,ITA,...,0.0,0.200000,12.909079,40.810707,3403015.75,3.860554e+06,2874068.0,49661,2016,6
48,6491527622,2016-06-16,16:48:17,POS,5411,0,0,30.17,T,ITA,...,0.0,0.125000,171.794554,28.940756,3403015.75,3.860554e+06,2874068.0,101,2016,6
50,6489132049,2016-06-15,21:23:07,INT,5735,0,6300,7.68,T,LUX,...,0.0,0.333333,17.025361,4.609863,3403015.75,3.860554e+06,2874068.0,125427,2016,6
51,6491520771,2016-06-16,16:46:36,POS,5411,0,0,30.17,T,ITA,...,0.0,0.166667,5.144011,28.984426,3403015.75,3.860554e+06,2874068.0,19587,2016,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5442901,6497210707,2016-06-18,15:51:41,ATM,5550,0,926100,0.00,T,ITA,...,0.0,0.000000,0.000000,0.000000,0.00,0.000000e+00,0.0,2,2016,6
5442902,6499220800,2016-06-19,15:28:30,ATM,6011,0,16100,20.00,T,ITA,...,0.0,0.000000,-1.000000,0.000000,0.00,0.000000e+00,0.0,84771,2016,6
5442903,6497533879,2016-06-18,17:15:33,ATM,5550,0,926100,0.00,T,ITA,...,0.0,0.000000,0.000000,0.000000,0.00,0.000000e+00,0.0,3,2016,6
5442904,6497533290,2016-06-18,17:15:30,ATM,5550,0,306100,0.00,T,ITA,...,0.0,0.000000,0.000000,0.000000,0.00,0.000000e+00,0.0,31104000,2016,6


In [14]:
# Normalizziamo la variabile importo

rob_scaler = RobustScaler()

df_test['IMP_TRZ_RECON_NORM'] = rob_scaler.fit_transform(df_test['IMP_TRZ_RECON'].values.reshape(-1,1)) 

df_test.drop(['IMP_TRZ_RECON'], axis=1, inplace=True)

c:\users\fap91\pycharmprojects\tesitirocinio\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\fap91\pycharmprojects\tesitirocinio\venv\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [15]:
#separating the X and the Y values
    
xTrain = df_train_undersample[['TIME_ORA_OP','ID_TIPO_CARTA','DEV_MEDIA_IMP_TC_1M', 'PERC_RISK_CIRC_TC_TO_1M',
       'DEV_MEDIA_IMP_CAN_1M', 'DEV_MEDIA_IMP_TO_1M', 'PERC_OPER_NON_AUTOR_1M',
       'PERC_OPER_NON_AUTOR_2M', 'MEDIA_NON_AUT_SEC_4',
       'DEV_IMP_MEDIA_ULTIME_3', 'PERC_RISK_MCC_1M', 'FLAG_IMP_PREC_0',
       'TREND_3_5','FLAG_CAMBIO_PAESE_NON_AUT',
       'PERC_ESITO_NSF_G', 'PERC_ESITO_CV2_G', 'PERC_ESITO_PIN_G',
       'PERC_ESITO_SCUDO_G', 'PERC_ESITO_DT_SCAD_G', 'PERC_ESITO_ALTRO_G',
       'DEV_MEDIA_AUT_SEC_4','IMP_TRZ_RECON_NORM']]

xTest = df_test[['TIME_ORA_OP','ID_TIPO_CARTA','DEV_MEDIA_IMP_TC_1M', 'PERC_RISK_CIRC_TC_TO_1M',
       'DEV_MEDIA_IMP_CAN_1M', 'DEV_MEDIA_IMP_TO_1M', 'PERC_OPER_NON_AUTOR_1M',
       'PERC_OPER_NON_AUTOR_2M', 'MEDIA_NON_AUT_SEC_4',
       'DEV_IMP_MEDIA_ULTIME_3', 'PERC_RISK_MCC_1M', 'FLAG_IMP_PREC_0',
       'TREND_3_5','FLAG_CAMBIO_PAESE_NON_AUT',
       'PERC_ESITO_NSF_G', 'PERC_ESITO_CV2_G', 'PERC_ESITO_PIN_G',
       'PERC_ESITO_SCUDO_G', 'PERC_ESITO_DT_SCAD_G', 'PERC_ESITO_ALTRO_G',
       'DEV_MEDIA_AUT_SEC_4','IMP_TRZ_RECON_NORM']]

yTrain = df_train_undersample["FLAG_FRODE"]


yTest = df_test["FLAG_FRODE"]

In [16]:
lr = LogisticRegression(random_state = 0, solver = 'liblinear', C= 1.0, penalty = 'l2',
                       n_jobs = -1)
lr.fit(xTrain,yTrain)
predictions_lr = lr.predict(xTest)
print(confusion_matrix(yTest, predictions_lr))

c:\users\fap91\pycharmprojects\tesitirocinio\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1357: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[[238105 467245]
 [  1754   4602]]


In [17]:
import numpy as np
cm_lr = confusion_matrix(yTest, predictions_lr)

tn = cm_lr[0][0]
fp = cm_lr[0][1]
fn = cm_lr[1][0]
tp = cm_lr[1][1]

recall = tp/(tp+fn)
specificity = tn/(tn+fp)

# sensitivity, hit rate, recall, or true positive rate
print('Accuracy',(tp+tn)/(tp+fp+fn+tn))

# sensitivity, hit rate, recall, or true positive rate
recall = recall_score(yTest, predictions_lr) 
print("The Recall is {}".format(recall)) 

# specificity or true negative rate
print('Specificity', tn/(tn+fp) )

#F1 Score
f1score = f1_score(yTest, predictions_lr) 
print("The F1 Score is {}".format(f1score)) 

#Precision
print("Precision: " ,tp/(tp+fp) )
precision = precision_score(yTest, predictions_lr) 
print("The precision Score is {}".format(precision)) 


roc_auc = roc_auc_score(yTest, predictions_lr) 
print("The ROC AUC Score is {}".format(roc_auc)) 

avg_prec = average_precision_score(yTest, predictions_lr) 
print("The Average precision Score is {}".format(avg_prec)) 

#Gmean
print('G-Mean',sqrt(recall*specificity))

Accuracy 0.3410214330074497
The Recall is 0.724040276903713
Specificity 0.33757000070886795
The F1 Score is 0.01924705616652342
Precision:  0.00975316151210032
The precision Score is 0.00975316151210032
The ROC AUC Score is 0.5308051388062904
The Average precision Score is 0.009526182552964734
G-Mean 0.49438272298659


In [18]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [6,8,10],
    'n_estimators': [100, 200, 300]
}
# Create a based model
xgb = XGBClassifier(random_state=0,n_jobs=-1,objective='binary:logistic', 
                    eval_metric = 'logloss',use_label_encoder =False)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = xgb, param_grid = param_grid, 
                          cv = 3)

In [ ]:
# Fit the grid search to the data
grid_search.fit(xTrain, yTrain)
grid_search.best_params_

In [ ]:
#For XGB
# Make predictions on validation dataset
XGB = XGBClassifier(random_state=0,n_jobs=-1,use_label_encoder =False, eval_metric = 'logloss')
XGB.fit(xTrain,yTrain)
predictions_xgb = XGB.predict(xTest)
print(accuracy_score(yTest, predictions_xgb))
print(confusion_matrix(yTest, predictions_xgb))
print(classification_report(yTest, predictions_xgb))

In [ ]:
cm_xgb = confusion_matrix(yTest, predictions_xgb)

tn = cm_xgb[0][0]
fp = cm_xgb[0][1]
fn = cm_xgb[1][0]
tp = cm_xgb[1][1]

recall = tp/(tp+fn)
specificity = tn/(tn+fp)

# sensitivity, hit rate, recall, or true positive rate
print('Accuracy',(tp+tn)/(tp+fp+fn+tn))

# sensitivity, hit rate, recall, or true positive rate
recall = recall_score(yTest, predictions_xgb) 
print("The Recall is {}".format(recall)) 

# specificity or true negative rate
print('Specificity', tn/(tn+fp) )

#F1 Score
f1score = f1_score(yTest, predictions_xgb) 
print("The F1 Score is {}".format(f1score)) 

#Precision
print("Precision: " ,tp/(tp+fp) )
precision = precision_score(yTest, predictions_xgb) 
print("The precision Score is {}".format(precision)) 


roc_auc = roc_auc_score(yTest, predictions_xgb) 
print("The ROC AUC Score is {}".format(roc_auc)) 

avg_prec = average_precision_score(yTest, predictions_xgb) 
print("The Average precision Score is {}".format(avg_prec)) 

#Gmean
print('G-Mean',sqrt(recall*specificity))


In [ ]:
#For XGB
# Make predictions on validation dataset
XGB = XGBClassifier(random_state=0,n_jobs=-1,objective='binary:logistic', 
                    max_depth = 6, n_estimators = 100, eval_metric = 'logloss',use_label_encoder =False)
XGB.fit(xTrain,yTrain)
predictions_xgb = XGB.predict(xTest)
print(accuracy_score(yTest, predictions_xgb))
print(confusion_matrix(yTest, predictions_xgb))
print(classification_report(yTest, predictions_xgb))

In [ ]:
cm_xgb = confusion_matrix(yTest, predictions_xgb)

tn = cm_xgb[0][0]
fp = cm_xgb[0][1]
fn = cm_xgb[1][0]
tp = cm_xgb[1][1]

recall = tp/(tp+fn)
specificity = tn/(tn+fp)

# sensitivity, hit rate, recall, or true positive rate
print('Accuracy',(tp+tn)/(tp+fp+fn+tn))

# sensitivity, hit rate, recall, or true positive rate
recall = recall_score(yTest, predictions_xgb) 
print("The Recall is {}".format(recall)) 

# specificity or true negative rate
print('Specificity', tn/(tn+fp) )

#F1 Score
f1score = f1_score(yTest, predictions_xgb) 
print("The F1 Score is {}".format(f1score)) 

#Precision
print("Precision: " ,tp/(tp+fp) )
precision = precision_score(yTest, predictions_xgb) 
print("The precision Score is {}".format(precision)) 


roc_auc = roc_auc_score(yTest, predictions_xgb) 
print("The ROC AUC Score is {}".format(roc_auc)) 

avg_prec = average_precision_score(yTest, predictions_xgb) 
print("The Average precision Score is {}".format(avg_prec)) 

#Gmean
print('G-Mean',sqrt(recall*specificity))


In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [6,8,10],
    'max_features': [2,3],
    'n_estimators': [100, 200, 300]
}
# Create a based model
rf = RandomForestClassifier(random_state=0)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
# Fit the grid search to the data
grid_search.fit(xTrain, yTrain)
grid_search.best_params_

In [ ]:
print (f'Train Accuracy - : {grid_search.score(xTrain,yTrain):.3f}')
print (f'Test Accuracy - : {grid_search.score(xValid,yValid):.3f}')

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [8,10,12],
    'max_features': [2,3,4],
    'n_estimators': [100, 200]
}
# Create a based model
rf = RandomForestClassifier(random_state=0)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
# Fit the grid search to the data
grid_search.fit(xTrain, yTrain)
grid_search.best_params_

In [ ]:
#building the Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier 
#random forest model creation 
rfc = RandomForestClassifier(bootstrap = True, random_state=0,
                             max_features = 2, max_depth = 12, n_estimators = 100)
rfc.fit(xTrain,yTrain)
predictions = rfc.predict(xTest)
print(accuracy_score(yTest, predictions))
print(confusion_matrix(yTest, predictions))
print(classification_report(yTest, predictions))

In [ ]:
import numpy as np
cm = confusion_matrix(yTest, predictions)

tn = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tp = cm[1][1]

recall = tp/(tp+fn)
specificity = tn/(tn+fp)

# sensitivity, hit rate, recall, or true positive rate
print('Accuracy',(tp+tn)/(tp+fp+fn+tn))

# sensitivity, hit rate, recall, or true positive rate
print('Recall',tp/(tp+fn))

# specificity or true negative rate
print('Specificity', tn/(tn+fp) )

#F1 Score
print("F1 Score: " ,tp/(tp+((fn+fp)/2)))

#Precision
print("Precision: " ,tp/(tp+fp) )

roc_auc = roc_auc_score(yTest, predictions) 
print("The ROC AUC Score is {}".format(roc_auc)) 

avg_prec = average_precision_score(yTest, predictions) 
print("The Average precision Score is {}".format(avg_prec)) 

#Gmean
print('G-Mean',sqrt(recall*specificity))

# precision or positive predictive value
ppv = tp/(tp+fp)
# negative predictive value
npv = tn/(tn+fn)
# fall out or false positive rate
fpr = fp/(fp+tn)
# false negative rate
fnr = fn/(tp+fn)
# false discovery rate
fdr = fp/(tp+fp)

# overall accuracy
acc = (tp+tn)/(tp+fp+fn+tn)

In [ ]:
print (f'Train Accuracy - : {grid_search.score(xTrain,yTrain):.3f}')
print (f'Test Accuracy - : {grid_search.score(xValid,yValid):.3f}')

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [10,12,14],
    'max_features': [2,3,4],
    'n_estimators': [100, 200, 300]
}
# Create a based model
rf = RandomForestClassifier(random_state=0)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
# Fit the grid search to the data
grid_search.fit(xTrain, yTrain)
grid_search.best_params_

In [ ]:
#building the Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier 
#random forest model creation 
rfc = RandomForestClassifier(bootstrap = True, random_state=0,
                             max_features = 4, max_depth = 14, n_estimators = 200)
rfc.fit(xTrain,yTrain)
predictions = rfc.predict(xTest)
print(accuracy_score(yTest, predictions))
print(confusion_matrix(yTest, predictions))
print(classification_report(yTest, predictions))

In [ ]:
import numpy as np
cm = confusion_matrix(yTest, predictions)

tn = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tp = cm[1][1]

recall = tp/(tp+fn)
specificity = tn/(tn+fp)

# sensitivity, hit rate, recall, or true positive rate
print('Accuracy',(tp+tn)/(tp+fp+fn+tn))

# sensitivity, hit rate, recall, or true positive rate
print('Recall',tp/(tp+fn))

# specificity or true negative rate
print('Specificity', tn/(tn+fp) )

#F1 Score
print("F1 Score: " ,tp/(tp+((fn+fp)/2)))

#Precision
print("Precision: " ,tp/(tp+fp) )

roc_auc = roc_auc_score(yTest, predictions) 
print("The ROC AUC Score is {}".format(roc_auc)) 

avg_prec = average_precision_score(yTest, predictions) 
print("The Average precision Score is {}".format(avg_prec)) 

#Gmean
print('G-Mean',sqrt(recall*specificity))

# precision or positive predictive value
ppv = tp/(tp+fp)
# negative predictive value
npv = tn/(tn+fn)
# fall out or false positive rate
fpr = fp/(fp+tn)
# false negative rate
fnr = fn/(tp+fn)
# false discovery rate
fdr = fp/(tp+fp)

# overall accuracy
acc = (tp+tn)/(tp+fp+fn+tn)

In [ ]:
rfc.get_params().keys()

In [ ]:
#building the Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier 
#random forest model creation 
rfc = RandomForestClassifier(bootstrap = True, random_state=0,
                             max_features = 2, max_depth = 14, n_estimators = 200)

rfc.fit(xTrain,yTrain)
predictions = rfc.predict(xTest)
print(accuracy_score(yTest, predictions))
print(confusion_matrix(yTest, predictions))
print(classification_report(yTest, predictions))

In [ ]:
import numpy as np
cm = confusion_matrix(yTest, predictions)

tn = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tp = cm[1][1]

recall = tp/(tp+fn)
specificity = tn/(tn+fp)

# sensitivity, hit rate, recall, or true positive rate
print('Accuracy',(tp+tn)/(tp+fp+fn+tn))

# sensitivity, hit rate, recall, or true positive rate
print('Recall',tp/(tp+fn))

# specificity or true negative rate
print('Specificity', tn/(tn+fp) )

#F1 Score
print("F1 Score: " ,tp/(tp+((fn+fp)/2)))

#Precision
print("Precision: " ,tp/(tp+fp) )

roc_auc = roc_auc_score(yTest, predictions) 
print("The ROC AUC Score is {}".format(roc_auc)) 

avg_prec = average_precision_score(yTest, predictions) 
print("The Average precision Score is {}".format(avg_prec)) 

#Gmean
print('G-Mean',sqrt(recall*specificity))

# precision or positive predictive value
ppv = tp/(tp+fp)
# negative predictive value
npv = tn/(tn+fn)
# fall out or false positive rate
fpr = fp/(fp+tn)
# false negative rate
fnr = fn/(tp+fn)
# false discovery rate
fdr = fp/(tp+fp)

# overall accuracy
acc = (tp+tn)/(tp+fp+fn+tn)

In [ ]:
#For easy
# Make predictions on validation dataset
from imblearn.ensemble import EasyEnsembleClassifier
easy = EasyEnsembleClassifier(random_state=0)

In [ ]:
easy.fit(xTrain,yTrain)
predictions_easy = easy.predict(xTest)
print(accuracy_score(yTest, predictions_easy))
print(confusion_matrix(yTest, predictions_easy))
print(classification_report(yTest, predictions_easy))

In [ ]:
cm = confusion_matrix(yTest, predictions_easy)

tn = cm[0][0]
fp = cm[0][1]
fn = cm[1][0]
tp = cm[1][1]

recall = tp/(tp+fn)
specificity = tn/(tn+fp)

# sensitivity, hit rate, recall, or true positive rate
print('Accuracy',(tp+tn)/(tp+fp+fn+tn))

# sensitivity, hit rate, recall, or true positive rate
recall = recall_score(yTest, predictions_easy, average='weighted') 
print("The Recall is {}".format(recall)) 

# specificity or true negative rate
print('Specificity', tn/(tn+fp) )

#F1 Score
f1score = f1_score(yTest, predictions_easy, average='weighted') 
print("The F1 Score is {}".format(f1score)) 

#Precision
print("Precision: " ,tp/(tp+fp) )

roc_auc = roc_auc_score(yTest, predictions_easy) 
print("The ROC AUC Score is {}".format(roc_auc)) 

roc_auc = roc_auc_score(yTest, easy.predict_proba(xTest)[:,1]) 
print("AUC probability {}".format(roc_auc)) 

roc_auc = roc_auc_score(yTest, easy.predict_proba(xTest)[:,1]) 
print("ROC AUC {}".format(sklearn.metrics.auc))

roc_auc = roc_auc_score(yTest,predictions_easy ) 
print("ROC AUC {}".format(sklearn.metrics.auc))

avg_prec = average_precision_score(yTest, predictions_easy) 
print("The Average precision Score is {}".format(avg_prec)) 

#Gmean
print('G-Mean',sqrt(recall*specificity))

In [ ]:
predictions_easy

In [ ]:
easy.predict_proba(xTest)[:,1]

In [ ]:
from sklearn.metrics import plot_roc_curve
ax = plt.gca()

LR_disp = plot_roc_curve(lr, xTest, yTest, ax=ax, alpha=0.8)
rfc_disp = plot_roc_curve(rfc, xTest, yTest, ax=ax, alpha=0.8)
xgb_disp = plot_roc_curve(XGB, xTest, yTest, ax=ax, alpha=0.8)
ada_disp = plot_roc_curve(easy, xTest, yTest, ax=ax, alpha=0.8)


plt.show()

In [ ]:
from sklearn.metrics import plot_precision_recall_curve
ax = plt.gca()

LR_disp = plot_precision_recall_curve(lr, xTest, yTest, ax=ax, alpha=0.8)
rfc_disp = plot_precision_recall_curve(rfc, xTest, yTest, ax=ax, alpha=0.8)
xgb_disp = plot_precision_recall_curve(XGB, xTest, yTest, ax=ax, alpha=0.8)
ada_disp = plot_precision_recall_curve(easy, xTest, yTest, ax=ax, alpha=0.8)

plt.show()

## thresholds

In [ ]:
from sklearn.metrics import precision_recall_curve
# predict probabilities
rfc_probs = rfc.predict_proba(xTest)
rfc_probs

In [ ]:
from sklearn.metrics import f1_score
from numpy import arange

# apply threshold to positive probabilities to create labels
def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype('int')

probs = rfc_probs[:,1]
# define thresholds
thresholds = arange(0, 1, 0.001)
# evaluate each threshold
scores = [f1_score(yTest, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

In [ ]:
from sklearn.metrics import f1_score
from numpy import arange

# apply threshold to positive probabilities to create labels
def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype('int')

probs = rfc_probs[:,1]
# define thresholds
thresholds = arange(0, 1, 0.001)
# evaluate each threshold
scores = [roc_auc_score(yTest, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, AucScore=%.5f' % (thresholds[ix], scores[ix]))

In [ ]:
from sklearn.metrics import f1_score
from numpy import arange

# apply threshold to positive probabilities to create labels
def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype('int')

probs = rfc_probs[:,0]
# define thresholds
thresholds = arange(0, 1, 0.001)
# evaluate each threshold
scores = [accuracy_score(yTest, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)
print('Threshold=%.3f, accuracy_score=%.5f' % (thresholds[ix], scores[ix]))

In [ ]:
rfc_probs

In [ ]:
probs

In [ ]:
from sklearn.metrics import roc_curve, auc
from numpy import argmax

preds = rfc_probs[:,1]
fpr, tpr, threshold = roc_curve(yTest, preds)
roc_auc = auc(fpr, tpr)

# get the best threshold
J = tpr - fpr
ix = argmax(J)
best_thresh = threshold[ix]
print('Best Threshold=%f' % (best_thresh))